In [2]:
!pip install hiclass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 282 kB/s 
     |████████████████████████████████| 8.8 MB 66.7 MB/s 
     |████████████████████████████████| 4.1 MB 72.2 MB/s 
     |████████████████████████████████| 144 kB 72.2 MB/s 
     |████████████████████████████████| 461 kB 72.0 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.46.3
    Uninstalling grpcio-1.46.3:
      Successfully uninstalled grpcio-1.46.3


In [3]:
!pip install mahotas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.7 MB 14.7 MB/s 


In [4]:
import mahotas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
from PIL import Image, ImageOps
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from hiclass import LocalClassifierPerNode
from hiclass import LocalClassifierPerParentNode
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, chi2

In [5]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


### **Load Images and Extract TAS Descriptors**

In [6]:
image_dir = "/gdrive/My Drive/Project/Denoised_CLAHE_Cl3"

In [7]:
def load_data(tag='train'):
  tag_dir = os.path.join(image_dir, tag)
  tag_path = Path(tag_dir)
  data = [] # Images
  cat = [] # Category
  subcat = []
  patient = []
  file_name = []
  for mag_dir in tag_path.iterdir():  
    mag_label = mag_dir.stem
    print("*",mag_label)
    if mag_label == "100X":
      print("Got it!")
      for img_name in mag_dir.glob('*.png'):
        img_label = img_name.stem
        splitted_image_name = img_label.split('_')
        cat_label = splitted_image_name[1]
        remaining_part = splitted_image_name[2].split('-')
        subcat_label = remaining_part[0]
        patient_label = remaining_part[2]
        img = mahotas.imread(img_name.as_posix())
        img = img[:, :, 0]
        feature = mahotas.features.pftas(img)
        hi_label = [cat_label, subcat_label]
        data.append(feature) # append the feature to the data
        cat.append(hi_label) # append the label to the category
        subcat.append(subcat_label)
        patient.append(patient_label)
        file_name.append(img_label)
  return data, cat, subcat, patient, file_name

In [8]:
# train images
vec_train, cat_train, subcat_train, patient_train, image_name_train = load_data('train')
# test images
vec_test, cat_test, subcat_test, patient_test, image_name_test = load_data('test')

* 40X
* 100X
Got it!
* 200X
* 400X
* 40X
* 100X
Got it!
* 200X
* 400X


### **Function to Compute Patient Recognition Rate**

In [9]:
def evaluate_recognition_rate(prediction):
  unique_patient_test = list(np.unique(np.array(patient_test)))
  num_images_per_patient = []
  num_correctly_classified = []
  num = 0
  total_patient_score = 0
  num_patient = 0
  for patient in unique_patient_test:
    num_patient = num_patient + 1
    Np = 0
    Nrec = 0
    image_index_counter = -1
    for image_name in image_name_test:
      image_index_counter = image_index_counter + 1
      image_name_splitted = image_name.split('-')
      if patient == image_name_splitted[2]:
        first_part_splitted = image_name_splitted[0].split('_')
        image_class = first_part_splitted[2]
        if image_class == prediction[image_index_counter]:
          Nrec = Nrec + 1
        Np = Np + 1
    patient_score = Nrec/Np
    #print(patient_score)
    num_images_per_patient.append(Np)
    num_correctly_classified.append(Nrec)
    total_patient_score = total_patient_score + patient_score
    num = num + Np    
  recognition_rate = total_patient_score/num_patient
  print("Summation of patient score: ", total_patient_score)
  print("Total Number of Patients: ", num_patient)
  print("Recognition Rate: ", recognition_rate)

### **Function to Identify Predicted Classes and Subclasses**

In [10]:
def return_predicted_categories(predictions):
  pred_column1 = []
  pred_column2 = []
  cat_test_column1 = []
  cat_test_column2 = []

  n = 0
  for row in predictions:
    pred_column1.append(predictions[n][0])
    pred_column2.append(predictions[n][1])
    cat_test_column1.append(cat_test[n][0])
    cat_test_column2.append(cat_test[n][1])
    n = n+1

  return pred_column1, pred_column2, cat_test_column1, cat_test_column2

### **SVM Fit**

In [11]:
pipe1 = Pipeline([
                 ("scale", StandardScaler()),
                 ("classifier", SVC(probability=True, kernel = 'rbf', gamma = 'auto', C = 10)) #radial basis function kernel
                 ])

In [12]:
classifier = LocalClassifierPerParentNode(local_classifier=pipe1)
classifier.fit(vec_train, cat_train)

LocalClassifierPerParentNode(local_classifier=Pipeline(steps=[('scale',
                                                               StandardScaler()),
                                                              ('classifier',
                                                               SVC(C=10,
                                                                   gamma='auto',
                                                                   probability=True))]))

**Accuracy**

In [13]:
predictions = classifier.predict(vec_test)
pred_column1, pred_column2, cat_test_column1, cat_test_column2 = return_predicted_categories(predictions)
print('Parent Class Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column1, cat_test_column1)))
print('Subclass Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column2, cat_test_column2)))

Parent Class Prediction accuracy: 81.84%
Subclass Prediction accuracy: 44.47%


**Patient Recognition Rate**

In [14]:
evaluate_recognition_rate(pred_column2)

Summation of patient score:  14.023764039971997
Total Number of Patients:  28
Recognition Rate:  0.5008487157132856


### **Random Forest**

In [15]:
pipe2 = Pipeline([
                 ("scale", StandardScaler()),
                 ("classifier", RandomForestClassifier(class_weight='balanced', n_estimators = 200, random_state = 42))
                 ])

In [16]:
classifier = LocalClassifierPerParentNode(local_classifier=pipe2)
classifier.fit(vec_train, cat_train)

LocalClassifierPerParentNode(local_classifier=Pipeline(steps=[('scale',
                                                               StandardScaler()),
                                                              ('classifier',
                                                               RandomForestClassifier(class_weight='balanced',
                                                                                      n_estimators=200,
                                                                                      random_state=42))]))

**Accuracy**

In [17]:
predictions = classifier.predict(vec_test)
pred_column1, pred_column2, cat_test_column1, cat_test_column2 = return_predicted_categories(predictions)
print('Parent Class Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column1, cat_test_column1)))
print('Subclass Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column2, cat_test_column2)))

Parent Class Prediction accuracy: 75.92%
Subclass Prediction accuracy: 43.03%


**Patient Recognition Rate**

In [18]:
evaluate_recognition_rate(pred_column2)

Summation of patient score:  13.829112655287819
Total Number of Patients:  28
Recognition Rate:  0.4938968805459935


### **KNN**

In [19]:
pipe3 = Pipeline([
                  ("select",SelectKBest(chi2,k=50)),
                 ("scale",StandardScaler()),
                 ("classifier", KNeighborsClassifier(n_neighbors=7))
                 ])

In [20]:
classifier = LocalClassifierPerParentNode(local_classifier=pipe3)
classifier.fit(vec_train, cat_train)

LocalClassifierPerParentNode(local_classifier=Pipeline(steps=[('select',
                                                               SelectKBest(k=50,
                                                                           score_func=<function chi2 at 0x7fb81cf90320>)),
                                                              ('scale',
                                                               StandardScaler()),
                                                              ('classifier',
                                                               KNeighborsClassifier(n_neighbors=7))]))

**Accuracy**

In [21]:
predictions = classifier.predict(vec_test)
pred_column1, pred_column2, cat_test_column1, cat_test_column2 = return_predicted_categories(predictions)
print('Parent Class Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column1, cat_test_column1)))
print('Subclass Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column2, cat_test_column2)))

Parent Class Prediction accuracy: 74.08%
Subclass Prediction accuracy: 39.74%


**Patient Recognition Rate**

In [22]:
evaluate_recognition_rate(pred_column2)

Summation of patient score:  12.727848493094067
Total Number of Patients:  28
Recognition Rate:  0.45456601761050236


### **Adaboost**

In [23]:
pipe4 = Pipeline([
                 ("scale",StandardScaler()),
                 ("classifier", AdaBoostClassifier(n_estimators=10, random_state=42))
                 ])

In [24]:
classifier = LocalClassifierPerParentNode(local_classifier=pipe4)
classifier.fit(vec_train, cat_train)

LocalClassifierPerParentNode(local_classifier=Pipeline(steps=[('scale',
                                                               StandardScaler()),
                                                              ('classifier',
                                                               AdaBoostClassifier(n_estimators=10,
                                                                                  random_state=42))]))

**Accuracy**

In [25]:
predictions = classifier.predict(vec_test)
pred_column1, pred_column2, cat_test_column1, cat_test_column2 = return_predicted_categories(predictions)
print('Parent Class Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column1, cat_test_column1)))
print('Subclass Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column2, cat_test_column2)))

Parent Class Prediction accuracy: 73.68%
Subclass Prediction accuracy: 42.11%


**Patient Recognition Rate**

In [26]:
evaluate_recognition_rate(pred_column2)

Summation of patient score:  12.869956251564
Total Number of Patients:  28
Recognition Rate:  0.4596412946987143


### **MLP**

In [36]:
mlp = MLPClassifier(alpha=0.1, max_iter=2000)

In [37]:
classifier = LocalClassifierPerParentNode(local_classifier=mlp)
classifier.fit(vec_train, cat_train)

LocalClassifierPerParentNode(local_classifier=MLPClassifier(alpha=0.1,
                                                            max_iter=2000))

**Accuracy**

In [38]:
predictions = classifier.predict(vec_test)
pred_column1, pred_column2, cat_test_column1, cat_test_column2 = return_predicted_categories(predictions)
print('Parent Class Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column1, cat_test_column1)))
print('Subclass Prediction accuracy: {:.2%}'.format(accuracy_score(pred_column2, cat_test_column2)))

Parent Class Prediction accuracy: 69.47%
Subclass Prediction accuracy: 42.63%


**Patient Recognition Rate**

In [39]:
evaluate_recognition_rate(pred_column2)

Summation of patient score:  13.736264960529667
Total Number of Patients:  28
Recognition Rate:  0.4905808914474881
